## CAP Theorem
The CAP Theorem says that it is impossible to build an implementation of read-write storage in an asynchronous network that satisfies all of the following three properties:
- **Availability:** will a request made to the data store always eventually complete?  A non-failing node will return a reasonable response within a reasonable amount of time (no error or timeout).
- **Consistency:** will all executions of reads and writes seen by all nodes be *atomic* or *linearizably* consistent? A read is guaranteed to return the most recent write for a given client.
- **Partition tolerance:** the network is allowed to drop any messages. The system will continue to function when network partitions occur.

CAP theorem tells us that we can't build a database that both responds to every request and returns the results that you would expect every time. Some elaboration on the terms used above:

**Read-write storage:** deals with construct called as *register* which is a data-structure supporting the following operations: `get()` and `set(X)`. Multiple registers can be combined together to form a key-value database.

**Atomic (or Linearizable):** under atomic consistency, multiple register operations potentially done by multiple clients concurrently, should be equivalent to single serial execution of all operations. The following may be true under eventual consistency, but not under atomic consistency:
```
set(10), set(5), get() = 10
```

Some other examples:
```
// Atomic consistency
B: set(5), A: set(10), A: get() = 10, B: get() = 10

// Not
B: set(5), A: set(10), A: get() = 10, B: get() = 5
```

## Availability
A data store is available if and only if all the `get` and `set` requests eventually return a response that's part of their specification. Error messages do not count, since any system can be trivially available by returning error in all cases.

The system can take time to respond, but it has to eventually respond. As per CAP theorem, if we choose availability over consistency (given that every systm must be partition tolerant), it looks like:

<img src="images/availability.png" >

### Availability Patterns
There are two complementary availability patterns:

**Fail Over** there are two variants: a) *Active-active* b) *Active-passive*. In active-passive, the active servers receive the all the requests, whereas the passive servers are on standby exchanging heartbeat with the active servers. When the active server goes down, passive ones take the responsibility and take over active server's IP. The length of downtime is determined by whether the passive server is already running in 'hot' standby or whether it needs to start up from 'cold' standby.

In active-active mode, both the servers are receiving traffic and the load balancer distributes the traffic amongst them.

**Replication** *master-slave* and *master-master* - more applicable to databases.

### Availability Statistics
Availability of a system is often expressed in terms of uptime (or downtime) as a percentage of time the service is available. It is common to see services have 99.9% availability. A service having 99.9% availability can afford to be down maximum of:
```
Duration                  Acceptable downtime
Downtime per year            8h 45min 57s
Downtime per month            43m 49.7s
Downtime per week             10m 4.8s
Downtime per day              1m 26.4s
```

**Combined availability:** two services A and B both have availability of 99.9%. What is the availability of the combined system when
1. Service A and B are invoked sequentially: $availablity(system) = availablity(A) \times availablity(B) = 0.999 \times 0.999 = 99.8%$
2. Service A and B are invoked parallely: $availablity(system) = 1 - (1 -availablity(A)) \times (1 - availablity(B)) = 1 - (1 - 0.999) \times (1 - 0.999) = 99.9999%$

## Consistency
In a consistent/partition toleran system, we wait for a response from the partitioned node which could result in a timeout error. The system can also choose to return an error, depending on the scenario we desire.

<img src="images/consistency.png" >

### Consistency Patterns
CAP theorem deals with strong consistency. However, there are some other consistency patterns that we may follow:\

**Weak consistency:** After a write, reads may or may not see it. A best effort approach is taken. Weak consistency works well in real time use cases such as VoIP, video chat, and realtime multiplayer games.

**Strong consistency:** After a write, reads will see it. Data is replicated synchronously. This approach is seen in file systems and RDBMSes. Strong consistency works well in systems that need transactions. This is the type of consistency, we talk about in CAP theorem.

**Eventual consistency:** After a write, reads will eventually see it (typically within milliseconds). Data is replicated asynchronously.